cubicasaのsvgから部屋を検出してマスクを作成。オリジナル画像とマスク画像の小さい方を基準にしてクロップ

In [ ]:
!pip install svg.path

In [ ]:
!curl -L -o cubicasa5k.zip\
  https://www.kaggle.com/api/v1/datasets/download/qmarva/cubicasa5k

In [ ]:
!unzip -q cubicasa5k.zip

In [ ]:
rm -r mask2former

display: noneでもスキップしないように変更

左上基準でクロップすると解決

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET
import numpy as np
import cv2
import re
import subprocess
from svg.path import parse_path
import shutil

def parse_transform(transform_str):
    """transform属性を3x3行列に変換"""
    m = re.search(r"matrix\(\s*([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)[,\s]+([-\d\.e]+)\s*\)", transform_str)
    if m:
        a, b, c, d, e, f = map(float, m.groups())
        return np.array([[a, c, e],
                         [b, d, f],
                         [0, 0, 1]])
    return np.eye(3)

def transform_point(x, y, T):
    """点(x, y)に変換行列Tを適用"""
    pt = np.array([x, y, 1])
    tpt = T @ pt
    return tpt[0], tpt[1]

def extract_room_masks(svg_file, output_file, output_color_file):
    """SVGファイルからroom_maskを抽出してPNG形式で保存"""
    tree = ET.parse(svg_file)
    root = tree.getroot()

    # SVGのサイズを取得。属性がなければスルー（処理を中断）
    if "width" not in root.attrib or "height" not in root.attrib:
        print(f"SVGのサイズが指定されていないため、スキップ: {svg_file}")
        return
    width = int(float(root.attrib["width"]))
    height = int(float(root.attrib["height"]))

    # マスクの初期化
    room_mask = np.zeros((height, width), dtype=np.uint8)
    room_instance_mask = np.zeros((height, width), dtype=np.uint16)
    instance_id = 1

    # 部屋とみなすクラスのキーワード
    room_keywords = {"Room", "LivingRoom", "Space"}

    def process_element(element, parent_class, parent_style, current_transform):
        nonlocal instance_id
        element_class = element.attrib.get("class", "")
        classes = element_class.split() if element_class else []
        parent_tokens = parent_class.split() if parent_class else []
        all_classes = set(classes) | set(parent_tokens)

        # "Room", "LivingRoom", "Space" を含むかどうか判定
        if not any(token in all_classes for token in room_keywords):
            return

        # display:none の要素はスキップしない
        style = element.attrib.get("style", "") + parent_style
        #if "display:none" in style.replace(" ", ""):
        #    return

        tag = element.tag.split('}')[-1]
        #print(f"Processing room element: {tag}, classes: {all_classes}")

        if tag == "path":
            d = element.attrib.get("d", "")
            if d:
                try:
                    path_obj = parse_path(d)
                    pts = []
                    for segment in path_obj:
                        seg_length = segment.length(error=1e-5)
                        num_samples = max(10, int(seg_length))
                        for t in np.linspace(0, 1, num_samples, endpoint=False):
                            pt = segment.point(t)
                            pts.append((float(pt.real), float(pt.imag)))

                    if pts:
                        if d.strip()[-1].upper() == "Z":
                            pts.append(pts[0])
                        else:
                            pt = path_obj[-1].point(1.0)
                            pts.append((float(pt.real), float(pt.imag)))

                        pts_transformed = [transform_point(x, y, current_transform) for (x, y) in pts]
                        pts_array = np.array(pts_transformed, dtype=np.int32)

                        fill_attr = element.attrib.get("fill", "")
                        if not (fill_attr and fill_attr.lower() == "none"):
                            cv2.fillPoly(room_mask, [pts_array], 255)
                            cv2.fillPoly(room_instance_mask, [pts_array], instance_id)
                            instance_id += 1
                except Exception as e:
                    print(f"Error processing path: {e}")

        elif tag == "polygon":
            points_str = element.attrib.get("points", "")
            try:
                point_pairs = points_str.strip().split()
                pts = []
                for pair in point_pairs:
                    coords = pair.strip().split(',')
                    if len(coords) >= 2:
                        pts.append((float(coords[0]), float(coords[1])))
                if pts:
                    pts_transformed = [transform_point(x, y, current_transform) for (x, y) in pts]
                    pts_array = np.array(pts_transformed, dtype=np.int32)
                    cv2.fillPoly(room_mask, [pts_array], 255)
                    cv2.fillPoly(room_instance_mask, [pts_array], instance_id)
                    instance_id += 1
            except Exception as e:
                print(f"Error processing polygon: {e}")

        elif tag == "rect":
            try:
                x = float(element.attrib.get("x", "0"))
                y = float(element.attrib.get("y", "0"))
                w = float(element.attrib.get("width", "0"))
                h = float(element.attrib.get("height", "0"))
                rect_pts = [(x, y), (x+w, y), (x+w, y+h), (x, y+h)]
                pts_transformed = [transform_point(px, py, current_transform) for (px, py) in rect_pts]
                pts_array = np.array(pts_transformed, dtype=np.int32)
                cv2.fillPoly(room_mask, [pts_array], 255)
                cv2.fillPoly(room_instance_mask, [pts_array], instance_id)
                instance_id += 1
            except Exception as e:
                print(f"Error processing rect: {e}")

    def process_group(group, current_transform, parent_style):
        style = group.attrib.get("style", "") + parent_style
        group_class = group.attrib.get("class", "")

        if "transform" in group.attrib:
            T = parse_transform(group.attrib["transform"])
            current_transform = current_transform @ T

        for child in group:
            tag = child.tag.split('}')[-1]
            if tag == "g":
                process_group(child, current_transform, style)
            else:
                process_element(child, group_class, style, current_transform)

    # ルート要素から処理開始
    identity = np.eye(3)
    for child in root:
        tag = child.tag.split('}')[-1]
        if tag == "g":
            process_group(child, identity, "")
        else:
            process_element(child, "", "", identity)

    # カラーマスクの生成
    color_mask = np.zeros((height, width, 3), dtype=np.uint8)
    unique_ids = np.unique(room_instance_mask)[1:]  # 0を除外

    # 各インスタンスに異なる色を割り当て
    np.random.seed(42)
    for uid in unique_ids:
        color = tuple(map(int, np.random.randint(0, 256, 3)))
        mask = room_instance_mask == uid
        color_mask[mask] = color

        # インスタンスIDを文字で重ねる例
        """
        y, x = np.where(mask)
        if len(x) > 0 and len(y) > 0:
            center_x = int(np.mean(x))
            center_y = int(np.mean(y))
            cv2.putText(color_mask, str(uid), (center_x, center_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        """

    # 結果を保存
    #cv2.imwrite(output_file, room_mask)
    cv2.imwrite(output_color_file, color_mask)
    #print(f"Saved room mask to: {output_file}")
    #print(f"Saved colored room mask to: {output_color_file}")

def svg_to_png_with_inkscape(input_svg, output_png):
    """
    Inkscape コマンドラインを使って SVG を PNG に変換するヘルパー関数。
    Inkscape 1.0 以降がインストールされている必要があります。
    """
    cmd = [
        "inkscape",
        "--export-type=png",
        f"--export-filename={output_png}",
        "--export-background=#ffffff",       # 背景色を白
        "--export-background-opacity=1.0",    # 不透明度を 1.0
        input_svg
    ]
    # subprocess.run() で外部コマンドを実行する
    subprocess.run(
        cmd,
        check=True,
        stdout=subprocess.DEVNULL,   # 必要なら標準出力も無視
        stderr=subprocess.DEVNULL    # 警告を含むエラー出力を非表示
    )

if __name__ == '__main__':
    import os
    import glob
    from tqdm import tqdm

    os.makedirs("mask2former", exist_ok=True)
    n = 0

    for base_dir in ["cubicasa5k/cubicasa5k/colorful", "cubicasa5k/cubicasa5k/high_quality"]:
        for i in tqdm(range(1, 20230)):
            # 該当するSVGファイルを検索
            # SVGファイルのパスを直接指定
            svg_file = os.path.join(base_dir, str(i), "model.svg")

            # ファイルが存在しない場合はスキップ
            if not os.path.exists(svg_file):
                continue
            ori_file = os.path.join(base_dir, str(i), "F1_scaled.png")

            n += 1
            shutil.copy(ori_file, f"mask2former/original_floorplan_{n}.png")

            # 出力ファイル名を設定
            output_dir = "mask2former/"
            os.makedirs(output_dir, exist_ok=True)

            output_file = f"{output_dir}/room_mask_{n}.png"
            output_color_file = f"{output_dir}/room_mask_color_{n}.png"

            # 部屋領域マスクを抽出
            extract_room_masks(svg_file, output_file, output_color_file)

            # SVG 全体を PNG 化 (Inkscape)
            #svg_to_png_with_inkscape(svg_file, output_original)
            """
            print(f"処理完了 - ディレクトリ {n}:")
            print(f"  SVGファイル: {svg_file}")
            print(f"  マスク出力: {output_file}")
            print(f"  カラーマスク出力: {output_color_file}")
            print(f"  オリジナルPNG出力: {output_original}")
            """


In [ ]:
import cv2
import numpy as np
import os
from tqdm import tqdm

def add_grid_noise(input_path, output_path,
                   line_spacing=40,      # 格子線の間隔（ピクセル）
                   line_thickness=1,    # 格子線の太さ
                   line_color=(0, 0, 0),# 格子線の色 (B, G, R)
                   alpha=1.0           # ノイズの透明度(0～1)
                  ):
    """
    画像に格子線ノイズを重ね合わせて保存する。
    - line_spacing: 格子線を引く間隔（水平・垂直方向）
    - line_thickness: 線の太さ
    - line_color: (B, G, R) で指定するカラー
    - alpha: 重ね合わせる格子線レイヤーの不透明度。0だと全く見えない、1だと完全に上書き。
    """
    img = cv2.imread(input_path)
    if img is None:
        print(f"Failed to load image: {input_path}")
        return

    h, w = img.shape[:2]

    # 元画像とは別に、線を引くためのオーバーレイ用画像を作成（コピー）
    overlay = img.copy()

    # 縦線を描画
    for x in range(0, w, line_spacing):
        cv2.line(overlay, (x, 0), (x, h), line_color, thickness=line_thickness)

    # 横線を描画
    for y in range(0, h, line_spacing):
        cv2.line(overlay, (0, y), (w, y), line_color, thickness=line_thickness)

    # アルファブレンドでノイズを重ね合わせる
    img_noisy = cv2.addWeighted(src1=overlay, alpha=alpha, src2=img, beta=(1 - alpha), gamma=0)

    cv2.imwrite(output_path, img_noisy)


def reduce_png_size_with_opencv(input_png, output_png, scale=None):
    """
    すでに作成済みの PNG を OpenCV で読み込み、指定されたスケールで縮小して保存。
    scaleが指定されていない場合は1000ピクセル以下になるまで75%ずつ縮小。
    """
    img = cv2.imread(input_png)
    if img is None:
        return None

    h, w = img.shape[:2]

    if scale is None:
        scale = 1.0
        # 高さか幅が1000を超える場合、1000以下になるまで75%ずつ縮小
        while h > 1000 or w > 1000:
            scale *= 0.75
            w = int(img.shape[1] * scale)
            h = int(img.shape[0] * scale)

    if scale < 1.0:
        w = int(img.shape[1] * scale)
        h = int(img.shape[0] * scale)
        resized = cv2.resize(img, (w, h), interpolation=cv2.INTER_NEAREST)
        cv2.imwrite(output_png, resized)
    else:
        # サイズが1000以下の場合はそのまま保存
        cv2.imwrite(output_png, img)

    return scale

# 実行例
if __name__ == "__main__":
    os.makedirs("mask2former_resize", exist_ok=True)

    for i in tqdm(range(1, 5001)):
        # 1. オリジナルのフロアプランに格子線ノイズを付与
        original_input = f"mask2former/original_floorplan_{i}.png"
        noisy_output   = f"mask2former_resize/original_floorplan_{i}.png"

        """
        add_grid_noise(original_input, noisy_output,
                       line_spacing=40,
                       line_thickness=1,
                       line_color=(0, 0, 0),  # 黒
                       alpha=1.0)
        """

        # 2. ノイズを付与した画像を縮小して保存 (上記で作成したnoisy_floorplan_{i}.pngを対象)
        scale = reduce_png_size_with_opencv(
            #input_png=noisy_output,
            input_png = original_input,
            output_png=noisy_output  # 上書きする場合。別ファイルにしたい場合はパスを変更
        )

        if scale is not None:
            # 同じスケールでroom_mask_colorも縮小
            input_png = f"mask2former/room_mask_color_{i}.png"
            output_png = f"mask2former_resize/room_mask_color_{i}.png"
            reduce_png_size_with_opencv(input_png, output_png, scale)


In [ ]:
import cv2
import os
from tqdm import tqdm

def align_resize_and_crop(input_dir, output_dir):
    """
    フロアプラン画像 (original_floorplan_{i}.png) と
    マスク画像 (room_mask_color_{i}.png) を以下の手順で処理する:
      1. 2つの画像の幅のうち、小さい方の幅に合わせて大きい方を縮小する。
         - 幅の比率を基準に各画像を縮小（小さい画像はそのまま）。
      2. 縮小後、両画像の高さが異なる場合は、左上を基準に
         高さが小さい方のサイズにクロップする。
    最終的に、両画像は同じサイズとなる。
    """
    os.makedirs(output_dir, exist_ok=True)

    for i in tqdm(range(1, 5001)):
        floorplan_path = os.path.join(input_dir, f"original_floorplan_{i}.png")
        mask_path      = os.path.join(input_dir, f"room_mask_color_{i}.png")

        if not (os.path.exists(floorplan_path) and os.path.exists(mask_path)):
            continue

        floorplan = cv2.imread(floorplan_path)
        mask_img  = cv2.imread(mask_path)

        if floorplan is None or mask_img is None:
            continue

        # 各画像の幅・高さを取得
        h_f, w_f = floorplan.shape[:2]
        h_m, w_m = mask_img.shape[:2]

        # 2つの画像のうち、小さい幅を基準にする
        target_width = min(w_f, w_m)

        # 小さい方を採用して左上基準でクロップ
        target_height = min(h_f, h_m)
        floorplan_cropped = floorplan[0:target_height, 0:target_width]
        mask_cropped      = mask_img[0:target_height, 0:target_width]

        # 出力ファイルパス
        out_floorplan = os.path.join(output_dir, f"original_floorplan_{i}.png")
        out_mask      = os.path.join(output_dir, f"room_mask_color_{i}.png")

        cv2.imwrite(out_floorplan, floorplan_cropped)
        cv2.imwrite(out_mask, mask_cropped)

if __name__ == "__main__":
    input_folder = "mask2former"        # 元画像が置いてあるフォルダ
    output_folder = "mask2former_resize" # 出力先フォルダ

    align_resize_and_crop(input_folder, output_folder)

In [ ]:
!zip -rq resize_without_noise.zip mask2former_resize

In [ ]:
!zip -rq mask2former.zip mask2former